# Plot raw data

**Date:** July 2019

**Purpose:** create a plot of the data pr cycle with voltage on one y-axis and current on the other where the individual steps are within each span with added labels showing C-rate and values from the step_table

In [ ]:
import os
import itertools

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import bokeh
import cellpy
from cellpy.utils import plotutils
from cellpy.readers.core import xldate_as_datetime

In [ ]:
%matplotlib inline
pd.options.display.max_columns = 100

In [ ]:
filename = "../testdata/hdf5/20160805_test001_45_cc.h5"
assert os.path.isfile(filename)
my_cell = cellpy.get(filename)

### Checking individual cycles using plot_raw_cycle

In [ ]:
my_cell.make_step_table()

In [ ]:
data = my_cell.dataset.raw
table = my_cell.dataset.steps
summary = my_cell.dataset.summary

In [ ]:
table.head(3)

In [ ]:
summary.head(3)

In [ ]:
data.head(3)

In [ ]:
my_cell.raw_limits

### Check using rawplot

In [ ]:
plotutils.raw_plot(my_cell)